# 신용카드 채무 불이행 고객 예측 Project 2

In [15]:
import pandas as pd

df = pd.read_csv('credit_card_pay.xlsx - Data.csv')
df.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
5,6,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,7,500000,1,1,2,29,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,8,100000,2,2,2,23,0,-1,-1,0,...,221,-159,567,380,601,0,581,1687,1542,0
8,9,140000,2,3,1,28,0,0,2,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0
9,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0


In [16]:
# row Data . LightGBM 모델 생성

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 데이터 불러오기
df = pd.read_csv('credit_card_pay.xlsx - Data.csv')


# 학습/검증 데이터 분리
X = df.drop(['ID', 'default payment next month'], axis=1)
y = df['default payment next month']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# LightGBM 모델 학습
params = {
    'learning_rate': 0.03,
    'max_depth': 8,
    'n_estimators': 500,
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'binary_logloss'
}

train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_val, label=y_val)
model = lgb.train(params, train_ds, 1000, val_ds, verbose_eval=10, early_stopping_rounds=100)

# feature importance 출력
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importance()})
feature_importance = feature_importance.sort_values(by='importance', ascending=False)
print(feature_importance)

C:\Users\GM220808\.venv\Practice1\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\GM220808\.venv\Practice1\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\GM220808\.venv\Practice1\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3262
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 23
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221208 -> initscore=-1.258639
[LightGBM] [Info] Start training from score -1.258639
Training until validation scores don't improve for 100 rounds
[10]	valid_0's binary_logloss: 0.482807
[20]	valid_0

In [17]:
from sklearn.metrics import classification_report

# 모델 예측
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)

# 예측 결과 출력 (정밀도, 재현율, F1-score)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4673
           1       0.69      0.38      0.49      1327

    accuracy                           0.82      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.82      0.80      6000



In [12]:
# pay_0의 -2, -1, 0 (무소비, 정액결제, 리볼빙으로 예상)을 모두 0값으로 치환 -> 모델의 학습과 결과적으로 -2, -1, 0 모두 채무를 이행하려는 의지로 보고 모두 0으로 치환

import numpy as np
# -2, -1, 0 -> 0
df['PAY_0'] = np.where((df['PAY_0']<=0), 0, df['PAY_0'])
df['PAY_2'] = np.where((df['PAY_2']<=0), 0, df['PAY_2'])
df['PAY_3'] = np.where((df['PAY_3']<=0), 0, df['PAY_3'])
df['PAY_4'] = np.where((df['PAY_4']<=0), 0, df['PAY_4'])
df['PAY_5'] = np.where((df['PAY_5']<=0), 0, df['PAY_5'])
df['PAY_6'] = np.where((df['PAY_6']<=0), 0, df['PAY_6'])
# (PAY_X 평균)/(신용한도/10000): 클수록 디폴트 가능성 높음. (default와 상관관계 0.100799)
# classification이므로 추후 변수제거 안해도 됨.
# df['delay/Limit_bal']=((df['PAY_0']+df['PAY_2']+df['PAY_3']+df['PAY_4']+df['PAY_5']+df['PAY_6'])/6)/(df['LIMIT_BAL']/10000)

In [ ]:
# 월별 청구금액 컬럼 추출
bill_amt_cols = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
bill_amt = df[bill_amt_cols]

# 월별 청구금액 증감률 계산   -> 청구금액이 급격하게 증가한 고객은 채무불이행할 가능성이 높다고 판단
bill_change = bill_amt.pct_change(axis=1)
bill_change = bill_change.replace([np.inf, -np.inf], np.nan).fillna(0)

# 청구금액 증감률 평균 계산
bill_change['bill_change_mean'] = bill_change.mean(axis=1)

# 데이터프레임에 추가
df = pd.concat([df, bill_change['bill_change_mean']], axis=1)

In [ ]:
# LightGBM

!pip install lightgbm

In [13]:
# LightGBM 모델 생성

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 데이터 불러오기
df = pd.read_csv('credit_card_pay.xlsx - Data.csv')

# 파생변수 추가
df['LIMIT_USE_AVG'] = (df['BILL_AMT1'] + df['BILL_AMT2'] + df['BILL_AMT3'] + df['BILL_AMT4'] + df['BILL_AMT5'] + df['BILL_AMT6']) / df['LIMIT_BAL']    # 신용한도에 따른 총 청구금액 비율
df['delay/Limit_bal']=((df['PAY_0']+df['PAY_2']+df['PAY_3']+df['PAY_4']+df['PAY_5']+df['PAY_6'])/6)/(df['LIMIT_BAL']/10000)                            # 신용한도에 따른 결제지연 기간의 총합
df['delay']=(df['PAY_0']+df['PAY_2']+df['PAY_3']+df['PAY_4']+df['PAY_5']+df['PAY_6'])/6                                                                # 결제 지연기간의 평균
bill_amt = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']]
bill_change = bill_amt.pct_change(axis=1)
bill_change = bill_change.replace([np.inf, -np.inf], np.nan).fillna(0)
bill_change['bill_change_mean'] = bill_change.mean(axis=1)

# 데이터프레임에 추가
df = pd.concat([df, bill_change['bill_change_mean']], axis=1)


# 학습/검증 데이터 분리
X = df.drop(['ID', 'default payment next month'], axis=1)
y = df['default payment next month']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# LightGBM 모델 학습
params = {
    'learning_rate': 0.03,
    'max_depth': 8,
    'n_estimators': 500,
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'binary_logloss'
}

train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_val, label=y_val)

model = lgb.train(params, train_ds, 1000, val_ds, verbose_eval=10, early_stopping_rounds=100)

# feature importance 출력
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importance()})
feature_importance = feature_importance.sort_values(by='importance', ascending=False)
print(feature_importance)

In [14]:
from sklearn.metrics import classification_report

# 모델 예측
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)

# 예측 결과 출력 (정밀도, 재현율, F1-score)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4673
           1       0.68      0.38      0.49      1327

    accuracy                           0.82      6000
   macro avg       0.76      0.66      0.69      6000
weighted avg       0.81      0.82      0.80      6000

